# Import The Packages

In [1]:
import os
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import imageio

In [2]:
from albumentations import HorizontalFlip, VerticalFlip, ElasticTransform, GridDistortion, OpticalDistortion, CoarseDropout

In [3]:
import matplotlib.pyplot as plt

In [6]:
from patchify import patchify

In [7]:
def load_data(path):
    """ X = Images and Y = masks """

    train_x = sorted(glob(os.path.join(path, "training", "images", "*.tif")))
    train_y = sorted(glob(os.path.join(path, "training", "1st_manual", "*.gif")))

    test_x = sorted(glob(os.path.join(path, "test", "images", "*.tif")))
    test_y = sorted(glob(os.path.join(path, "test", "1st_manual", "*.gif")))

    return (train_x, train_y), (test_x, test_y)

In [8]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [76]:
def augment_data(images, masks, save_path, augment=True):

    count = 1
    for idx, (x, y) in tqdm(enumerate(zip(images, masks)), total=len(images)):
        """ Extracting names """
        name = x.split("\\")[-1].split(".")[0]
        

        """ Reading image and mask """
        x = cv2.imread(x, cv2.IMREAD_COLOR)
        y = imageio.mimread(y)[0]

        top, bottom = [28] * 2
        left, right = [37,38]
        
        x = cv2.copyMakeBorder(x, top, bottom, left, right, cv2.BORDER_CONSTANT, value= 0)
        y = cv2.copyMakeBorder(y, top, bottom, left, right, cv2.BORDER_CONSTANT, value= 0)
        
#         print(x.shape)
#         patches_img = patchify(x, (128, 128,3), step=128)
#         print(len(patches_img))
#         print(patches_img[0][0].shape)
#         plt.imshow(patches_img[0][0])
#         break
    
        aug = HorizontalFlip(p=1.0)
        augmented = aug(image=x, gt=y)
        x1 = augmented["image"]
        y1 = augmented["gt"]

        aug = VerticalFlip(p=1.0)
        augmented = aug(image=x, gt=y)
        x2 = augmented["image"]
        y2 = augmented["gt"]

        aug = ElasticTransform(p=1, alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03)
        augmented = aug(image=x, gt=y)
        x3 = augmented['image']
        y3 = augmented['gt']

        aug = GridDistortion(p=1)
        augmented = aug(image=x, gt=y)
        x4 = augmented['image']
        y4 = augmented['gt']

        aug = OpticalDistortion(p=1, distort_limit=2, shift_limit=0.5)
        augmented = aug(image=x, gt=y)
        x5 = augmented['image']
        y5 = augmented['gt']

        X = [x, x1, x2, x3, x4, x5]
        Y = [y, y1, y2, y3, y4, y5]


        index = 0
        for i, m in zip(X, Y):

#             image_path = os.path.join(save_path, "image", tmp_image_name)
#             mask_path = os.path.join(save_path, "mask", tmp_mask_name)

#             cv2.imwrite(image_path, i)
#             cv2.imwrite(mask_path, m)

            patches_img = patchify(x, (128, 128,3), step=128)
            patches_gt = patchify(y, (128, 128), step=128)
        
        
            for i in range(patches_img.shape[0]):
                for j in range(patches_img.shape[1]):
                    if i == 0:
                        if j == 0 or j == 4:
                            continue
                    elif i == 4:
                        if j == 0 or j == 4:
                            continue
                    
                    single_patch_img = patches_img[i,j,:,:][0]
                    tmp_image_name = f"{name}_{index}_{i}{j}.jpg" 
                    
                    single_patch_gt = patches_gt[i,j,:,:]
                    tmp_gt_name = f"{name}_{index}_{i}{j}.jpg"
                    
                    if count % 4 == 0:
                        fol = "valid\\"
                    else:
                        fol = "train\\"
                        
                    image_path = os.path.join(save_path,fol,"image", tmp_image_name)
                    cv2.imwrite(image_path, single_patch_img)
                    gt_path = os.path.join(save_path, fol, "gt", tmp_gt_name)
                    cv2.imwrite(gt_path, single_patch_gt)
                    
                    count += 1
                    
            index += 1

In [39]:
""" Seeding """
np.random.seed(42)

""" Load the data """
data_path = "D:\\Data sets\\DRIVE"
(train_x, train_y), (test_x, test_y) = load_data(data_path)

print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Test: {len(test_x)} - {len(test_y)}")


Train: 20 - 20
Test: 20 - 0


In [79]:
""" Creating directories """
create_dir("D:\\new_data_128\\train\\image")
create_dir("D:\\new_data_128\\train\\gt")
create_dir("D:\\new_data_128\\valid\\image")
create_dir("D:\\new_data_128\\valid\\gt")

In [80]:
augment_data(train_x, train_y, "D:\\new_data_128\\", augment=True)

100%|██████████| 20/20 [00:05<00:00,  3.70it/s]


In [42]:
### Shift 16 images from train to valid

In [39]:
# augment_data(test_x, test_y, "new_data/test/", augment=False)

In [62]:
set([4,4])

{4}